# Welcome to the dataquality client demo

### This will be a brief introduction to how the client works and a bit under the hood

## Installing

You can currently install dataquality from pypi
`pip install dataquality`

But for development, you may want to install it from github. This will give you the latest changes in master

`pip install git+https://www.github.com/rungalileo/dataquality.git`

You can also clone the repo and install from a path. This is recommended for development

`pip install /path/to/dataquality/directory`

**(It's good to restart the kernel after an install)**

In [ ]:
!pip install -q /path/to/dataquality

In [ ]:
# If you have cloned the dataquality repo and are running this from the tests/notebooks folder, you can run this
!pip install -q ../../../dataquality

In [ ]:
!pip install -qqq git+https://www.github.com/rungalileo/dataquality.git

## Components

The data quality client is currently very simple. It has just a few components:

* logging - the inputs and outputs to your model
* config - the urls, usernames, and passwords to interact with the server
* init - how you start a new project/run
* finish - how you end your run

## Getting started

To get started, simply `import dataquality`<br>
If your environment variables are set, your import will pass through. If not, you will be prompted for some url and config variables.<br>

To bypass the prompt, set the following environment variables
* `GALILEO_API_URL`
* `GALILEO_MINIO_URL`
* `GALILEO_MINIO_ACCESS_KEY`
* `GALILEO_MINIO_SECRET_KEY`
* `GALILEO_MINIO_REGION`

If you have your server (api, minio, mysql) running locally for development, the following will work
```
import os

os.environ['GALILEO_API_URL']="http://localhost:8000"
os.environ['GALILEO_MINIO_URL']="127.0.0.1:9000"
os.environ['GALILEO_MINIO_ACCESS_KEY']="minioadmin"
os.environ['GALILEO_MINIO_SECRET_KEY']="minioadmin"
```

If you don't set these environment variables, the client will prompt you for the fields (assuming you're running from the newest code).

### How do I get everything running locally??

See our [CONTRIBUTING](https://github.com/rungalileo/api/blob/main/CONTRIBUTING.md) doc
(When running the API, use the `./scripts/run-gunicorn.sh` - you don't need all of them)

In [ ]:
import os

os.environ['GALILEO_API_URL']="http://localhost:8088"
os.environ['GALILEO_MINIO_URL']="127.0.0.1:9000"
os.environ['GALILEO_MINIO_ACCESS_KEY']="minioadmin"
os.environ['GALILEO_MINIO_SECRET_KEY']="minioadmin"
!rm .galileo/config.json

In [ ]:
# For dev

# import os
# os.environ['GALILEO_API_URL']="http://api.dev.rungalileo.io"
# os.environ['GALILEO_MINIO_URL']="data.dev.rungalileo.io"
# os.environ['GALILEO_MINIO_ACCESS_KEY']="minioadmin"
# os.environ['GALILEO_MINIO_SECRET_KEY']="minioadmin"
# !rm .galileo/config.json

In [ ]:
import dataquality

## Logging in

Once you have dataquality imported, you can log into your server and start logging data<br>

To log in, you can call `dataquality.login()` <br>
This will prompt you for your auth method, email, and password. You can skip this prompt with the following environment variables:

* `GALILEO_AUTH_METHOD`
* `GALILEO_USERNAME`
* `GALILEO_PASSWORD`

### How do I create a user?

If you are running everything locally, you can do the following

```
import requests

data={
  "email": "me@rungalileo.io",
  "first_name": "Me",
  "last_name": "Me",
  "username": "Galileo",
  "auth_method": "email",
  "password": "Th3secret_"
}

r = requests.post('http://localhost:8088/users', json=data)
r.json()
```

Then set your env vars
```
import os

os.environ["GALILEO_AUTH_METHOD"]="email"
os.environ["GALILEO_USERNAME"]="{r.json()['email']}"
os.environ["GALILEO_PASSWORD"]="{r.json()['password']}"
```

If you don't set these environment variables, the client will prompt you for the fields (assuming you're running from the newest code).

Now login

```
dataquality.login()
```


In [ ]:
import requests

pwd = "MyPassword!123"

data={
  "email": "me@rungalileo.io",
  "first_name": "Me",
  "last_name": "Me",
  "username": "Galileo",
  "auth_method": "email",
  "password": pwd
}

r = requests.post('http://localhost:8088/users', json=data)
r.json()

import os

os.environ["GALILEO_AUTH_METHOD"]="email"
os.environ["GALILEO_USERNAME"]=f"{r.json()['email']}"
os.environ["GALILEO_PASSWORD"]=pwd

In [ ]:
dataquality.login()

## Start my project/run

Now you can start using the tool with `dataquality.init()`<br>

You can optionally provide a project name for this run

In [ ]:
dataquality.init?

In [ ]:
# Base case
dataquality.init()

In [ ]:
# New project, unset run (new)
dataquality.init(project_name="a_new_project")

In [ ]:
# Existing project, unset run (new)
dataquality.init(project_name="a_new_project")

In [ ]:
# Existing project, new run
dataquality.init(project_name="a_new_project", run_name="a_new_run")

In [ ]:
# Existing project, existing run
dataquality.init(project_name="a_new_project", run_name="a_new_run")

In [ ]:
# New project, new run
dataquality.init(project_name="a_new_project2", run_name="a_new_run2")

## Log to my project/run

Now that you've started your run, all you need to do is log data to it.<br>

To make that easier, we've created the `GalileoModelConfig` and `GalileoDataConfig` classes that you can log with. They have smart validators that will tell you if something is wrong.<br>

Your `GalileoDataConfig` takes your text, label, and (optional) id fields. If ids aren't provided, we will make them for you.<br>

Your `GalileoModelConfig` takes your embedding, probability, and id fields.

In [ ]:
from dataquality.core.integrations.config import GalileoModelConfig, GalileoDataConfig

help(GalileoDataConfig)

In [ ]:
help(GalileoModelConfig)

### Use them to log data

We use the `log_batch_input_data` and `log_model_outputs` to log our metadata

In [ ]:
dataquality.init()

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

newsgroups = fetch_20newsgroups(subset="train", remove=('headers', 'footers', 'quotes'))

dataset = pd.DataFrame()
dataset["text"] = newsgroups.data
label_ind = newsgroups.target_names
dataset["label"] = [label_ind[i] for i in newsgroups.target]
dataset = dataset[:100]

data_conf_train = GalileoDataConfig(text=dataset['text'], labels=dataset['label'], split="train")
data_conf_test = GalileoDataConfig(text=dataset['text'], labels=dataset['label'], split="test")

dataquality.log_batch_input_data(data_conf_train)
dataquality.log_batch_input_data(data_conf_test)

## We validate data before logging

#### See what happens with an invalid model config

In [ ]:
# Labels and text inputs dont match in shape
bad_data_conf = GalileoDataConfig(text=dataset['text'], labels=dataset['label'][:3], split="train")

dataquality.log_batch_input_data(bad_data_conf)

In [ ]:
import numpy as np

# Generate fake model outputs
def log_fake_data(log_num: int = 0):
    # Ensure unique IDs
    # Because we're going to call this twice, we need the other dataset rows for the second call, so /2
    num_rows = len(dataset) // 2 
        
    emb = np.random.rand(num_rows, 800)
    prob = np.random.rand(num_rows, 20)
    for split in ['test','train']:
        epoch = 0
        
        r = range(num_rows*log_num, num_rows*(log_num+1))
        ids = list(r)
        model_conf = GalileoModelConfig(emb=emb, probs=prob, split=split, epoch=epoch, ids=ids)
        dataquality.log_model_outputs(model_conf)

log_fake_data()

In [ ]:
!tree .galileo/logs/{dataquality.config.current_project_id}/{dataquality.config.current_run_id}

### What happened?

When you call `log_batch_input_data` you are logging the input data for this training job. This would typically be run once (per split).<br>

Then, as you train your model in batches, each call to `log_model_outputs` takes the data in that batch, joins it to the input data, and stores it in 3 files, data, emb, and prob.<br>

If we were to log another fake dataset to this, we'd see another file in each dir (under the epoch we set).

The file names in each subdir will match so we can join them at the end

In [ ]:
log_fake_data(1)

In [ ]:
!tree .galileo/logs/{dataquality.config.current_project_id}/{dataquality.config.current_run_id}

In [ ]:
import vaex
vaex.open(f'.galileo/logs/{dataquality.config.current_project_id}/{dataquality.config.current_run_id}/test/0/data/*.arrow')


## How do I see my results in the UI?

Simply set your labels (`set_labels_for_run`) and call `finish()`

Once called, the data will be joined together at a _per-epoch_ level, and added to minio, with one file for each `prob`, `emb`, and `data` per split/epoch. 

A job will be kicked off to process you data on the server, and after it's done you'll see your results inv the UI

#### Why do I need to set my labels?

Since your model is simply outputting probabilities, we have no way to map the index of each prediction to the model output. Setting your labels enables us to map them so you can see the meaningful values in the UI.<br>

If you have the UI running, you should see it at the URL returned.

**Note:** Check out your local API logs to see the background job!

In [ ]:
dataquality.set_labels_for_run(newsgroups.target_names)
dataquality.finish()

## That should take ~10-20 seconds to complete (if you are running the server locally)

### Now we can read the data from local minio

In [ ]:
from minio import Minio

client = Minio('localhost:9000', 'minioadmin', 'minioadmin', secure=False)
p = dataquality.config.current_project_id
r = dataquality.config.current_run_id
client.fget_object('galileo-project-runs-results', f'{p}/{r}/training/data/data.hdf5', 'training_data.hdf5')
client.fget_object('galileo-project-runs-results', f'{p}/{r}/test/data/data.hdf5', 'test_data.hdf5')

display(vaex.open('training_data.hdf5'))

display(vaex.open('test_data.hdf5'))